<h1 align=center><font size = 10>Improving Montevideo</font></h1>

## Introduction

The Government of Montevideo is interested in improving the city and the quality of live of its inhabitants, and also make it more attractive for small and large business. Ideally they want to do as much of the three improvements without doing one at the expense of another.

The perceptions is that there are areas more in need of improvements than others and the satisfaction of the citizens might be a good indicator that can be used as a way to identify those areas.
As there recently has been an election and the results are now available, this poll can be used as and indicator of satisfaction; where the Government has obtained a low share of the votes in relation to the opposition, it would be taken as an area where more needs to be done for the satisfaction of its inhabitants. 

The idea of this project is then to compare areas, in terms of type and number of venues and services, and also include the poll results as an’ indicator of citizen’s satisfaction.

## Approach

In order to achieve the required analysis we'll use data available from Montevideo city websites as well as the Electoral Department. This will then be integrated
with Foursquare data, obtained via API, then merged, segmented and clustered.
The data has been curated into two main data sources. For more detailed information see the Project document 
https://github.com/DSS-TAJorge/Coursera_Capstone/blob/master/Improving%20Montevideo.pdf

## 1. Import required libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 2. Import data and explore it

In [2]:
dfs = pd.read_excel('VotosMontevideo.xlsx')
print(dfs.head())
dfs.shape

   Circuito SERIE Municipio  Habilitados  No_Obs  Obs  Emitidos  EnBlanco  \
0         1   AAA         B          389     302    8       310         4   
1         2   AAA         B          389     344    4       348         7   
2         3   AAA         B          389     333    2       335         8   
3         4   AAA         B          389     317    2       319         4   
4         5   AAA         B          389     338    2       340         1   

   Anulados  PorSi   FA  PN  PC  PI  AP  PT  PE  PG  PV  PD  CA Desde  \
0         3      4  117  87  38   5   2   0   5   4   2   0  31     1   
1         8      3  129  81  23   6   4   0   6   2   2   0  73  6092   
2        10      1  124  67  35   5   4   1   3   2   5   0  68  7224   
3        10      2  133  62  20   5   3   0   3   6   1   0  68  8036   
4         5      3  164  78  26   4   5   0   2   1   5   1  43  9789   

     Hasta                                        Local  \
0   6091.0                 Centro SUB O

(2642, 26)




#### Let's remove unwanted data and check new shape

In [3]:
# Municipio is Z is not a physical localation - remove id
dfs = dfs[dfs.Municipio != 'Z']
dfs.shape

(2615, 26)

#### Group Data by Municipio adding the total votes by party

In [4]:
# get votes by Municipality 
municipio_votes=dfs.groupby(['Municipio'])['FA','PN','PC','PI','AP','PT','PE','PG','PV','PD','CA'].agg('sum').reset_index()
municipio_votes

,Municipio,FA,PN,PC,PI,AP,PT,PE,PG,PV,PD,CA
0,A,84472,25658,7204,1364,1744,134,2047,2068,1274,286,9792
1,B,49126,28264,14607,1829,1624,95,2167,961,1450,485,8620
2,C,55013,27337,13171,1701,1617,107,2432,1177,1491,456,8960
3,D,58034,26336,7980,1322,1124,89,1678,2051,1089,297,13757
4,E,43811,28034,17490,1589,1129,89,2191,986,1363,438,7017
5,F,53004,24749,6681,1229,979,54,1427,2125,1025,307,12040
6,G,52538,20349,7171,1211,1221,89,1735,1481,1078,282,7723
7,X,40247,32422,23507,2088,1348,54,2156,810,1238,494,6567


#### Now find the Government percentage of votes by municipality

In [5]:
# Obtain percentage of Government votes (FA) vs Opposition (PN + PC + PI + AP + PT + PE + PG + PV + PD + CA)
municipio_votes['FA_Percentage']=municipio_votes['FA']/municipio_votes.iloc[:,0:12].sum(axis=1)

In [6]:
# Add total votes to make sure is coherent
municipio_votes['Total_Votes']=municipio_votes.iloc[:,0:12].sum(axis=1)
municipio_votes

,Municipio,FA,PN,PC,PI,AP,PT,PE,PG,PV,PD,CA,FA_Percentage,Total_Votes
0,A,84472,25658,7204,1364,1744,134,2047,2068,1274,286,9792,0.620921,136043
1,B,49126,28264,14607,1829,1624,95,2167,961,1450,485,8620,0.449756,109228
2,C,55013,27337,13171,1701,1617,107,2432,1177,1491,456,8960,0.484858,113462
3,D,58034,26336,7980,1322,1124,89,1678,2051,1089,297,13757,0.510158,113757
4,E,43811,28034,17490,1589,1129,89,2191,986,1363,438,7017,0.420705,104137
5,F,53004,24749,6681,1229,979,54,1427,2125,1025,307,12040,0.511523,103620
6,G,52538,20349,7171,1211,1221,89,1735,1481,1078,282,7723,0.553743,94878
7,X,40247,32422,23507,2088,1348,54,2156,810,1238,494,6567,0.362811,110931


**First interesting finding: there is a clear difference between municipalities, the extremes are Municipio A and Municipio X  
Government has 62% of approval in the former and only 36% in the latter.**

#### Now import municipality data and check it

In [7]:
# Import Municipios data
dfm = pd.read_excel('MunicipiosAddresses.xlsx')
dfm

,Municipio,Address,Latitude,Longitude
0,A,Carlos María Ramírez 862,-34.865730,-56.235827
1,B,Joaquín Requena 1701,-34.897304,-56.171140
2,C,L. A. de Herrera 4547,-34.867148,-56.171454
3,X,Brito del Pino 1590,-34.899690,-56.156650
4,D,Av. Gral. Flores 4694,-34.845525,-56.154466
5,E,Estadio Charrúa,-34.878506,-56.089357
6,F,Av. 8 de Octubre 4700,-34.858786,-56.133496
7,G,Cno. Castro 730,-34.859575,-56.213453


### Visualise Montevideo and its municipalities

In [8]:
# get Mntevideo's Lat and Long
address = 'Montevideo, Uruguay'

geolocator = Nominatim(user_agent="montevideo")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of montevideo are {}, {}.'.format(latitude, longitude))

The geographical coordinate of montevideo are -34.9059039, -56.1913569.


In [9]:
# create map of montevideo using latitude and longitude values
map_montevideo = folium.Map(location=[latitude, longitude], zoom_start=12)
map_montevideo 

In [10]:
# add municipality centres to map

for lat, lng, municip in zip(dfm['Latitude'], dfm['Longitude'], dfm['Municipio']):
    label = '{}'.format(municip)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.1,
        parse_html=False).add_to(map_montevideo)  

map_montevideo

### Given the difference in Municipalities lets explore the venues in Municipalities A (index 0) and X (index 3)

#### Municipio A

In [45]:
# @hidden_cells
# credentials
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [12]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 3000 # define radius
municipio_latitude=dfm.loc[0, 'Latitude']
municipio_longitude=dfm.loc[0,'Longitude']
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    municipio_latitude, 
    municipio_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
# Find categories
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.io.json.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print("Categories in Municipio A:")
nearby_venues.head()

Categories in Municipio A:


,name,categories,lat,lng
0,Rosedal,Garden,-34.859560,-56.205701
1,Campomar F5,Soccer Field,-34.864487,-56.215044
2,La fortaleza,Pizza Place,-34.874959,-56.250936
3,Circulo De Tenis,Tennis Court,-34.862401,-56.205919
4,Parque José Nasazzi,Soccer Stadium,-34.861640,-56.208767


#### Municipio X

In [15]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
municipio_latitude=dfm.loc[3, 'Latitude']
municipio_longitude=dfm.loc[3,'Longitude']
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    municipio_latitude, 
    municipio_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [16]:
# Find categories
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.io.json.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print("Categories in Municipio X:")
nearby_venues.head()

Categories in Municipio X:


,name,categories,lat,lng
0,Parque Batlle,Park,-34.898597,-56.156181
1,Parrillada Feliciano,BBQ Joint,-34.899649,-56.153281
2,Sophilabs HQ,IT Services,-34.897425,-56.160791
3,Ulpiano,Restaurant,-34.901611,-56.160266
4,Velódromo Municipal,Track Stadium,-34.897758,-56.155868


**We have now found an interesting difference in categories between Municipio A and X: Municipio A has more sport and open spaces. We'll now carry the study and find out if that relation is confirmed.**

## 3. Prepare Segmentation and Clustering

Since Montevideo has a 200sq km area, each of the 8 municipalities cover approx 25sq km that is a radius of approx 3000 meters

In [17]:
# function to find nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=3000):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print('Getting data for Municipio {}'.format(name))
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Municipio', 
                  'Municipio Latitude', 
                  'Municipio Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the function on each neighborhood and create a new dataframe called *municipio_venues*.

In [18]:
municipio_venues = getNearbyVenues(names=dfm['Municipio'],
                                   latitudes=dfm['Latitude'],
                                   longitudes=dfm['Longitude']
                                  )

Getting data for Municipio A
Getting data for Municipio B
Getting data for Municipio C
Getting data for Municipio X
Getting data for Municipio D
Getting data for Municipio E
Getting data for Municipio F
Getting data for Municipio G


In [19]:
# check data
print(municipio_venues.shape)
municipio_venues.head()

(498, 7)


,Municipio,Municipio Latitude,Municipio Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,A,-34.86573,-56.235827,Rosedal,-34.859560,-56.205701,Garden
1,A,-34.86573,-56.235827,Campomar F5,-34.864487,-56.215044,Soccer Field
2,A,-34.86573,-56.235827,La fortaleza,-34.874959,-56.250936,Pizza Place
3,A,-34.86573,-56.235827,Circulo De Tenis,-34.862401,-56.205919,Tennis Court
4,A,-34.86573,-56.235827,Parque José Nasazzi,-34.861640,-56.208767,Soccer Stadium


In [20]:
# Check how many venues were returned for each Municipio

municipio_venues.groupby('Municipio').count()

,Municipio Latitude,Municipio Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Municipio,,,,,,
A,17,17,17,17,17,17
B,100,100,100,100,100,100
C,68,68,68,68,68,68
D,33,33,33,33,33,33
E,83,83,83,83,83,83
F,32,32,32,32,32,32
G,65,65,65,65,65,65
X,100,100,100,100,100,100


#### Let's find out how many unique categories from all the returned venues

In [21]:
print('There are {} uniques categories.'.format(len(municipio_venues['Venue Category'].unique())))

There are 109 uniques categories.


<a id='item3'></a>

## 4. Analyse Municipalities and Segment

In [22]:
# one hot encoding
municipio_onehot = pd.get_dummies(municipio_venues[['Venue Category']], prefix="", prefix_sep="")

# add municipio column back to dataframe
municipio_onehot['Municipio'] = municipio_venues['Municipio'] 

# move municipio column to the first column
fixed_columns = [municipio_onehot.columns[-1]] + list(municipio_onehot.columns[:-1])
municipio_onehot = municipio_onehot[fixed_columns]

municipio_onehot.head()

,Municipio,American Restaurant,Art Museum,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Big Box Store,Bistro,Bookstore,Brewery,Burger Joint,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hostel,Hot Dog Joint,Hotel,Housing Development,IT Services,Ice Cream Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Laboratory,Latin American Restaurant,Market,Mediterranean Restaurant,Middle Eastern Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Other Great Outdoors,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pub,Restaurant,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Toy / Game Store,Waterfront,Women's Store
0,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
municipio_onehot.shape

(498, 110)

#### Group rows by municipio and by taking the mean of the frequency of occurrence of each category

In [24]:
municipio_grouped = municipio_onehot.groupby('Municipio').mean().reset_index()
municipio_grouped

,Municipio,American Restaurant,Art Museum,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Big Box Store,Bistro,Bookstore,Brewery,Burger Joint,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hostel,Hot Dog Joint,Hotel,Housing Development,IT Services,Ice Cream Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Laboratory,Latin American Restaurant,Market,Mediterranean Restaurant,Middle Eastern Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Other Great Outdoors,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pub,Restaurant,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Toy / Game Store,Waterfront,Women's Store
0,A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.058824,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.058824,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.058824,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00000,0.000000,0.058824,0.000000,0.00,0.058824,0.000000,0.000000,0.00,0.000000,0.058824,0.000000,0.000000,0.000000,0.00,0.00,0.117647,0.176471,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000
1,B,0.000000,0.010000,0.010000,0.040000,0.020000,0.070000,0.010000,0.000000,0.000000,0.01,0.01,0.000000,0.01,0.010000,0.030000,0.01,0.000000,0.01,0.000000,0.01,0.050000,0.02,0.01,0.010000,0.000000,0.030000,0.000000,0.010000,0.010000,0.01,0.01,0.020000,0.010000,0.01,0.000000,0.000000,0.01,0.01,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.020000,0.000000,0.01,0.010000,0.010000,0.00000,0.010000,0.010000,0.01,0.000000,0.010000,0.00,0.01,0.000000,0.010000,0.020000,0.010000,0.010000,0.01,0.000000,0.000000,0.01,0.020000,0.000000,0.01,0.000000,0.01,0.020000,0.00000,0.010000,0.010000,0.010000,0.03,0.060000,0.000000,0.010000,0.01,0.010000,0.000000,0.000000,0.000000,0.000000,0.01,0.01,0.010000,0.030000,0.01,0.01,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000
2,C,0.000000,0.014706,0.044118,0.058824,0.044118,0.014706,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.014706,0.00,0.000000,0.00,0.000000,0.00,0.029412,0.00,0.00,0.029412,0.000000,0.000000,0.014706,0.000000,0.014706,0.00,0.00,0.014706,0.014706,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.029412,0.000000,0.014706,0.044118,0.014706,0.000000,0.000000,0.014706,0.044118,0.014706,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.044118,0.00,0.014706,0.014706,0.00,0.00,0.000000,0.014706,0.029412,0.014706,0.014706,0.00,0.000000,0.000000,0.00,0.014706,0.029412,0.00,0.014706,0.00,0.014706,0.00000,0.029412,0.088235,0.000000,0.00,0.044118,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0

Confirm the new size

In [25]:
municipio_grouped.shape

(8, 110)

#### Print each municipio along with the top 5 most common venues

In [26]:
num_top_venues = 5

for muni in municipio_grouped['Municipio']:
    print("----"+muni+"----")
    temp = municipio_grouped[municipio_grouped['Municipio'] == muni].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----A----
            venue  freq
0  Soccer Stadium  0.18
1          Garden  0.12
2    Soccer Field  0.12
3     Bus Station  0.06
4     Supermarket  0.06


----B----
            venue  freq
0             Bar  0.07
1      Restaurant  0.06
2     Coffee Shop  0.05
3       BBQ Joint  0.04
4  Soccer Stadium  0.03


----C----
            venue  freq
0     Pizza Place  0.09
1       BBQ Joint  0.06
2       Gastropub  0.04
3             Gym  0.04
4  Ice Cream Shop  0.04


----D----
         venue  freq
0  Supermarket  0.09
1   Restaurant  0.06
2    BBQ Joint  0.06
3  Pizza Place  0.06
4     Pharmacy  0.06


----E----
          venue  freq
0         Plaza  0.10
1   Pizza Place  0.10
2    Restaurant  0.06
3  Soccer Field  0.06
4      Pharmacy  0.04


----F----
               venue  freq
0        Pizza Place  0.12
1         Restaurant  0.06
2     Ice Cream Shop  0.06
3  Convenience Store  0.06
4       Soccer Field  0.06


----G----
            venue  freq
0       BBQ Joint  0.06
1          Bakery 

#### Put it into a dataframe

Function to sort the venues in descending order.

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Display the top 10 venues for each neighborhood.

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Municipio']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
municipio_venues_sorted = pd.DataFrame(columns=columns)
municipio_venues_sorted['Municipio'] = municipio_grouped['Municipio']

for ind in np.arange(municipio_grouped.shape[0]):
    municipio_venues_sorted.iloc[ind, 1:] = return_most_common_venues(municipio_grouped.iloc[ind, :], num_top_venues)

municipio_venues_sorted

,Municipio,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A,Soccer Stadium,Garden,Soccer Field,Basketball Court,Pizza Place,Restaurant,Shipping Store,Bus Station,Basketball Stadium,Historic Site
1,B,Bar,Restaurant,Coffee Shop,BBQ Joint,Deli / Bodega,Pub,Brewery,Soccer Stadium,Comfort Food Restaurant,Sushi Restaurant
2,C,Pizza Place,BBQ Joint,Gastropub,Gym,Restaurant,Soccer Field,Ice Cream Shop,Supermarket,Athletics & Sports,Bakery
3,D,Supermarket,Pharmacy,BBQ Joint,Coffee Shop,Pizza Place,Restaurant,Soccer Field,Department Store,Cosmetics Shop,Convenience Store
4,E,Plaza,Pizza Place,Soccer Field,Restaurant,Bakery,Shopping Mall,Beach,Park,Pharmacy,Supermarket
5,F,Pizza Place,Soccer Field,Ice Cream Shop,Convenience Store,Pharmacy,Restaurant,Grocery Store,Bakery,Supermarket,Steakhouse
6,G,Garden,BBQ Joint,Bakery,Gastropub,Gym,Ice Cream Shop,Supermarket,Soccer Field,Soccer Stadium,Pizza Place
7,X,Restaurant,Hotel,Bar,Coffee Shop,Soccer Stadium,Park,Pub,Café,Electronics Store,Other Great Outdoors


<a id='item4'></a>

## 5. Cluster Municipalities

#### Run *k*-means to cluster the municipio into 3 clusters.

In [29]:
# set number of clusters
kclusters = 3

municipio_grouped_clustering = municipio_grouped.drop('Municipio', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(municipio_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 0, 0, 2, 0, 0, 2], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each municipio.

In [30]:
# add clustering labels
municipio_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

municipio_merged = dfm

# merge municipio_grouped with municipio_data to add latitude/longitude for each neighborhood
municipio_merged = municipio_merged.join(municipio_venues_sorted.set_index('Municipio'), on='Municipio')

municipio_merged.head() 

,Municipio,Address,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A,Carlos María Ramírez 862,-34.865730,-56.235827,1,Soccer Stadium,Garden,Soccer Field,Basketball Court,Pizza Place,Restaurant,Shipping Store,Bus Station,Basketball Stadium,Historic Site
1,B,Joaquín Requena 1701,-34.897304,-56.171140,2,Bar,Restaurant,Coffee Shop,BBQ Joint,Deli / Bodega,Pub,Brewery,Soccer Stadium,Comfort Food Restaurant,Sushi Restaurant
2,C,L. A. de Herrera 4547,-34.867148,-56.171454,0,Pizza Place,BBQ Joint,Gastropub,Gym,Restaurant,Soccer Field,Ice Cream Shop,Supermarket,Athletics & Sports,Bakery
3,X,Brito del Pino 1590,-34.899690,-56.156650,2,Restaurant,Hotel,Bar,Coffee Shop,Soccer Stadium,Park,Pub,Café,Electronics Store,Other Great Outdoors
4,D,Av. Gral. Flores 4694,-34.845525,-56.154466,0,Supermarket,Pharmacy,BBQ Joint,Coffee Shop,Pizza Place,Restaurant,Soccer Field,Department Store,Cosmetics Shop,Convenience Store


Visualize the resulting clusters

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(municipio_merged['Latitude'], municipio_merged['Longitude'], municipio_merged['Municipio'], municipio_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

**Interesting, the cluster are linearly separable which implies a geographical correlation too**

## 6. Examine Clusters

#### Cluster 1

In [32]:
municipio_merged.loc[municipio_merged['Cluster Labels'] == 0, municipio_merged.columns[[1] + list(range(5, municipio_merged.shape[1]))]]

,Address,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,L. A. de Herrera 4547,Pizza Place,BBQ Joint,Gastropub,Gym,Restaurant,Soccer Field,Ice Cream Shop,Supermarket,Athletics & Sports,Bakery
4,Av. Gral. Flores 4694,Supermarket,Pharmacy,BBQ Joint,Coffee Shop,Pizza Place,Restaurant,Soccer Field,Department Store,Cosmetics Shop,Convenience Store
6,Av. 8 de Octubre 4700,Pizza Place,Soccer Field,Ice Cream Shop,Convenience Store,Pharmacy,Restaurant,Grocery Store,Bakery,Supermarket,Steakhouse
7,Cno. Castro 730,Garden,BBQ Joint,Bakery,Gastropub,Gym,Ice Cream Shop,Supermarket,Soccer Field,Soccer Stadium,Pizza Place


#### Cluster 2

In [33]:
municipio_merged.loc[municipio_merged['Cluster Labels'] == 1, municipio_merged.columns[[1] + list(range(5, municipio_merged.shape[1]))]]

,Address,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Carlos María Ramírez 862,Soccer Stadium,Garden,Soccer Field,Basketball Court,Pizza Place,Restaurant,Shipping Store,Bus Station,Basketball Stadium,Historic Site


#### Cluster 3

In [34]:
municipio_merged.loc[municipio_merged['Cluster Labels'] == 2, municipio_merged.columns[[1] + list(range(5, municipio_merged.shape[1]))]]

,Address,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Joaquín Requena 1701,Bar,Restaurant,Coffee Shop,BBQ Joint,Deli / Bodega,Pub,Brewery,Soccer Stadium,Comfort Food Restaurant,Sushi Restaurant
3,Brito del Pino 1590,Restaurant,Hotel,Bar,Coffee Shop,Soccer Stadium,Park,Pub,Café,Electronics Store,Other Great Outdoors
5,Estadio Charrúa,Plaza,Pizza Place,Soccer Field,Restaurant,Bakery,Shopping Mall,Beach,Park,Pharmacy,Supermarket


**The type of venues in the cluster clearly shows Cluster 1 as the most balanced, Cluster 2 shows more open spaces and sport venues, Cluster 3 more places for entertainment**

#### Name the cluster by their characteristics

In [35]:
# We'll add a column to identify the clusters by their characteristics to make it easier to analysis at a glance
municipio_merged.insert(5,'Cluster Name','')

In [36]:
municipio_merged.loc[municipio_merged['Cluster Labels']==0,'Cluster Name']='Balanced'
municipio_merged.loc[municipio_merged['Cluster Labels']==1,'Cluster Name']='OpenSpaces'
municipio_merged.loc[municipio_merged['Cluster Labels']==2,'Cluster Name']='Entertainment'

In [37]:
# Show final dataframe
municipio_merged

,Municipio,Address,Latitude,Longitude,Cluster Labels,Cluster Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A,Carlos María Ramírez 862,-34.865730,-56.235827,1,OpenSpaces,Soccer Stadium,Garden,Soccer Field,Basketball Court,Pizza Place,Restaurant,Shipping Store,Bus Station,Basketball Stadium,Historic Site
1,B,Joaquín Requena 1701,-34.897304,-56.171140,2,Entertainment,Bar,Restaurant,Coffee Shop,BBQ Joint,Deli / Bodega,Pub,Brewery,Soccer Stadium,Comfort Food Restaurant,Sushi Restaurant
2,C,L. A. de Herrera 4547,-34.867148,-56.171454,0,Balanced,Pizza Place,BBQ Joint,Gastropub,Gym,Restaurant,Soccer Field,Ice Cream Shop,Supermarket,Athletics & Sports,Bakery
3,X,Brito del Pino 1590,-34.899690,-56.156650,2,Entertainment,Restaurant,Hotel,Bar,Coffee Shop,Soccer Stadium,Park,Pub,Café,Electronics Store,Other Great Outdoors
4,D,Av. Gral. Flores 4694,-34.845525,-56.154466,0,Balanced,Supermarket,Pharmacy,BBQ Joint,Coffee Shop,Pizza Place,Restaurant,Soccer Field,Department Store,Cosmetics Shop,Convenience Store
5,E,Estadio Charrúa,-34.878506,-56.089357,2,Entertainment,Plaza,Pizza Place,Soccer Field,Restaurant,Bakery,Shopping Mall,Beach,Park,Pharmacy,Supermarket
6,F,Av. 8 de Octubre 4700,-34.858786,-56.133496,0,Balanced,Pizza Place,Soccer Field,Ice Cream Shop,Convenience Store,Pharmacy,Restaurant,Grocery Store,Bakery,Supermarket,Steakhouse
7,G,Cno. Castro 730,-34.859575,-56.213453,0,Balanced,Garden,BBQ Joint,Bakery,Gastropub,Gym,Ice Cream Shop,Supermarket,Soccer Field,Soccer Stadium,Pizza Place


## 7. Merge with votes to determine satisfaction

In [38]:
municipio_merged = pd.merge(municipio_merged,municipio_votes[['Municipio','FA_Percentage']],on='Municipio', how='left')

In [39]:
municipio_merged

,Municipio,Address,Latitude,Longitude,Cluster Labels,Cluster Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,FA_Percentage
0,A,Carlos María Ramírez 862,-34.865730,-56.235827,1,OpenSpaces,Soccer Stadium,Garden,Soccer Field,Basketball Court,Pizza Place,Restaurant,Shipping Store,Bus Station,Basketball Stadium,Historic Site,0.620921
1,B,Joaquín Requena 1701,-34.897304,-56.171140,2,Entertainment,Bar,Restaurant,Coffee Shop,BBQ Joint,Deli / Bodega,Pub,Brewery,Soccer Stadium,Comfort Food Restaurant,Sushi Restaurant,0.449756
2,C,L. A. de Herrera 4547,-34.867148,-56.171454,0,Balanced,Pizza Place,BBQ Joint,Gastropub,Gym,Restaurant,Soccer Field,Ice Cream Shop,Supermarket,Athletics & Sports,Bakery,0.484858
3,X,Brito del Pino 1590,-34.899690,-56.156650,2,Entertainment,Restaurant,Hotel,Bar,Coffee Shop,Soccer Stadium,Park,Pub,Café,Electronics Store,Other Great Outdoors,0.362811
4,D,Av. Gral. Flores 4694,-34.845525,-56.154466,0,Balanced,Supermarket,Pharmacy,BBQ Joint,Coffee Shop,Pizza Place,Restaurant,Soccer Field,Department Store,Cosmetics Shop,Convenience Store,0.510158
5,E,Estadio Charrúa,-34.878506,-56.089357,2,Entertainment,Plaza,Pizza Place,Soccer Field,Restaurant,Bakery,Shopping Mall,Beach,Park,Pharmacy,Supermarket,0.420705
6,F,Av. 8 de Octubre 4700,-34.858786,-56.133496,0,Balanced,Pizza Place,Soccer Field,Ice Cream Shop,Convenience Store,Pharmacy,Restaurant,Grocery Store,Bakery,Supermarket,Steakhouse,0.511523
7,G,Cno. Castro 730,-34.859575,-56.213453,0,Balanced,Garden,BBQ Joint,Bakery,Gastropub,Gym,Ice Cream Shop,Supermarket,Soccer Field,Soccer Stadium,Pizza Place,0.553743


In [41]:
municipio_summary=municipio_merged[['Municipio','Cluster Name','FA_Percentage']].sort_values(by=['FA_Percentage'])

In [42]:
municipio_summary

,Municipio,Cluster Name,FA_Percentage
3,X,Entertainment,0.362811
5,E,Entertainment,0.420705
1,B,Entertainment,0.449756
2,C,Balanced,0.484858
4,D,Balanced,0.510158
6,F,Balanced,0.511523
7,G,Balanced,0.553743
0,A,OpenSpaces,0.620921


**There is a correlation between the Cluster Type and the satisfaction measured by percentage of votes for the Government**